In [1]:
%pwd

'/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/src/analysis/round4'

In [2]:
%cd ../../../data/round4/raw

/Users/IvanTang/quant/IMC_Prosperity3_GraniteFlow/data/round4/raw


In [30]:
%ls

CROISSANTS.csv                   VOLCANIC_ROCK.csv
DJEMBES.csv                      VOLCANIC_ROCK_VOUCHER_10000.csv
JAMS.csv                         VOLCANIC_ROCK_VOUCHER_10250.csv
KELP.csv                         VOLCANIC_ROCK_VOUCHER_10500.csv
MAGNIFICENT_MACARONS.csv         VOLCANIC_ROCK_VOUCHER_9500.csv
PICNIC_BASKET1.csv               VOLCANIC_ROCK_VOUCHER_9750.csv
PICNIC_BASKET2.csv               mid_price_day1.csv
RAINFOREST_RESIN.csv             mid_price_day2.csv
SQUID_INK.csv                    mid_price_day3.csv


In [4]:
import pandas as pd
import numpy as np

In [11]:
df0 = pd.read_csv('VOLCANIC_ROCK.csv')
df1 = pd.read_csv('VOLCANIC_ROCK_VOUCHER_9500.csv')
df2 = pd.read_csv('VOLCANIC_ROCK_VOUCHER_9750.csv')
df3 = pd.read_csv('VOLCANIC_ROCK_VOUCHER_10000.csv')
df4 = pd.read_csv('VOLCANIC_ROCK_VOUCHER_10250.csv')
df5 = pd.read_csv('VOLCANIC_ROCK_VOUCHER_10500.csv')


In [16]:
def orderbook_preprocess(df):
    """
    对订单簿数据进行预处理
    1.填充NAN为0
    2.计算best_bid, best_ask, spread, mid_price, vwap, orderbook_imbalance, ask_bid_ratio
    """
    df = df.copy()
    #填充orderbook数据,nan填为0
    columns = ['ask_price_1', 'ask_volume_1', 'ask_price_2', 'ask_volume_2', 'ask_price_3', 'ask_volume_3', 'bid_price_1', 'bid_volume_1', 'bid_price_2', 'bid_volume_2', 'bid_price_3', 'bid_volume_3']
    for column in columns:
        df[column] = df[column].fillna(0)
    
    #计算best_bid, best_ask, spread, mid_price, vwap
    df['best_bid'] = df[['bid_price_1', 'bid_price_2', 'bid_price_3']].min(axis=1)
    df['best_ask'] = df[['ask_price_1', 'ask_price_2', 'ask_price_3']].max(axis=1)

    #计算best_bid, best_ask

    def get_best_bid_ask_spread(row):
    # 获取有效的买价（去掉NaN） 计算最佳买价
        bid_prices = [row['bid_price_1'], row['bid_price_2'], row['bid_price_3']]
        bid_prices = [price for price in bid_prices if price != 0]
        best_bid = min(bid_prices) if bid_prices else None

        # 获取有效的卖价（去掉NaN） 计算最佳卖价
        ask_prices = [row['ask_price_1'], row['ask_price_2'], row['ask_price_3']]
        ask_prices = [price for price in ask_prices if price != 0]
        best_ask = max(ask_prices) if ask_prices else None
        
        # 计算价差
        spread = best_ask - best_bid if best_ask and best_bid else None
        return best_bid, best_ask, spread

    df[['best_bid', 'best_ask','spread']] = df.apply(get_best_bid_ask_spread, axis=1, result_type='expand')

    #计算mid_price 
    df['mid_price'] = (df['ask_price_1']+ df['bid_price_1']) / 2

    #计算vwap
    df['vwap'] = (df['ask_price_1']*df['ask_volume_1'] + df['ask_price_2']*df['ask_volume_2'] + df['ask_price_3']*df['ask_volume_3'] + df['bid_price_1']*df['bid_volume_1'] + df['bid_price_2']*df['bid_volume_2'] + df['bid_price_3']*df['bid_volume_3']) / (df['ask_volume_1'] + df['ask_volume_2'] + df['ask_volume_3'] + df['bid_volume_1'] + df['bid_volume_2'] + df['bid_volume_3'])
    
    #计算orderbook_imbalance
    def orderbook_imbalance(row):
        """ 计算订单簿不平衡度 """
        buy_orders = [(row['bid_price_1'], row['bid_volume_1']), (row['bid_price_2'], row['bid_volume_2']), (row['bid_price_3'], row['bid_volume_3'])]
        sell_orders = [(row['ask_price_1'], row['ask_volume_1']), (row['ask_price_2'], row['ask_volume_2']), (row['ask_price_3'], row['ask_volume_3'])]
        buy_pressure = sum(amount for price, amount in buy_orders if price != 0)
        sell_pressure = sum(amount for price, amount in sell_orders if price != 0)
        total_pressure = buy_pressure + sell_pressure
        if total_pressure == 0:
            return 0
        return (buy_pressure - sell_pressure) / total_pressure
    
    df['orderbook_imbalance'] = df.apply(orderbook_imbalance, axis=1)

    #计算ask_bid_ratio
    df['ask_volume'] = df['ask_volume_1'] + df['ask_volume_2'] + df['ask_volume_3']
    df['bid_volume'] = df['bid_volume_1'] + df['bid_volume_2'] + df['bid_volume_3']
    df['ask_bid_ratio'] = (df['ask_volume'] - df['bid_volume']) / (df['ask_volume'] + df['bid_volume'])
    

    return df
    

In [17]:
df_list = [df0, df1, df2, df3, df4, df5]
for df in df_list:
    df = orderbook_preprocess(df)

In [20]:
df0

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,...,mid_price,profit_and_loss,best_bid,best_ask,spread,vwap,orderbook_imbalance,ask_volume,bid_volume,ask_bid_ratio
0,1,0,VOLCANIC_ROCK,10515.0,143.0,10514.0,57.0,0.0,0.0,10517,...,10516.0,0.0,10514.0,10517.0,3.0,10515.857500,0.000000,200.0,200.0,0.000000
1,1,100,VOLCANIC_ROCK,10514.0,104.0,10513.0,44.0,0.0,0.0,10516,...,10515.0,0.0,10513.0,10517.0,4.0,10515.000000,0.000000,148.0,148.0,0.000000
2,1,200,VOLCANIC_ROCK,10514.0,133.0,10513.0,62.0,0.0,0.0,10516,...,10515.0,0.0,10513.0,10517.0,4.0,10515.000000,0.000000,195.0,195.0,0.000000
3,1,300,VOLCANIC_ROCK,10516.0,200.0,0.0,0.0,0.0,0.0,10518,...,10517.0,0.0,10516.0,10519.0,3.0,10517.127500,0.000000,200.0,200.0,0.000000
4,1,400,VOLCANIC_ROCK,10519.0,108.0,10518.0,63.0,0.0,0.0,10520,...,10519.5,0.0,10518.0,10521.0,3.0,10519.492582,0.014837,166.0,171.0,-0.014837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,3,999500,VOLCANIC_ROCK,9915.0,141.0,9914.0,66.0,0.0,0.0,9917,...,9916.0,0.0,9914.0,9917.0,3.0,9915.840580,0.000000,207.0,207.0,0.000000
29996,3,999600,VOLCANIC_ROCK,9916.0,119.0,9915.0,75.0,0.0,0.0,9918,...,9917.0,0.0,9915.0,9918.0,3.0,9916.806701,0.000000,194.0,194.0,0.000000
29997,3,999700,VOLCANIC_ROCK,9917.0,115.0,9916.0,53.0,0.0,0.0,9918,...,9917.5,0.0,9916.0,9919.0,3.0,9917.500000,0.000000,168.0,168.0,0.000000
29998,3,999800,VOLCANIC_ROCK,9918.0,127.0,9917.0,46.0,0.0,0.0,9920,...,9919.0,0.0,9917.0,9920.0,3.0,9918.836795,0.026706,164.0,173.0,-0.026706


In [ ]:
df_mid = pd.concat([df0[['timestamp', 'day', 'mid_price']], df1['mid_price'], df2['mid_price'], df3['mid_price'], df4['mid_price'], df5['mid_price']], axis=1)

In [27]:
new_columns = ['timestamp', 'day', 
               'VOLCANIC_ROCK', 'VOLCANIC_ROCK_VOUCHER_9500', 'VOLCANIC_ROCK_VOUCHER_9750', 
               'VOLCANIC_ROCK_VOUCHER_10000', 'VOLCANIC_ROCK_VOUCHER_10250', 'VOLCANIC_ROCK_VOUCHER_10500']
df_mid = df_mid.set_axis(new_columns, axis=1)

In [29]:
df_mid_1 = df_mid[df_mid['day'] == 1]
df_mid_2 = df_mid[df_mid['day'] == 2]
df_mid_3 = df_mid[df_mid['day'] == 3]

df_mid_1.to_csv('mid_price_day1.csv', index=False)
df_mid_2.to_csv('mid_price_day2.csv', index=False)
df_mid_3.to_csv('mid_price_day3.csv', index=False)